In [6]:
import pandas as pd
import os
from pathlib import Path

# Đường dẫn gốc
BASE_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord")

# Định nghĩa các file cần gộp
FILES_CONFIG = {
    "Amazon": {
        "train": BASE_PATH / "Amazon" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Amazon" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Amazon" / "test_reviews_under80words.csv"
    },
    "Hotels": {
        "train": BASE_PATH / "Hotels" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Hotels" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Hotels" / "test_reviews_under80words.csv"
    },
    "AmazonsElectronics": {
        "all": BASE_PATH / "AmazonsElectronics" / "Kaggle" / "neutral_output_aspect_filledNone.csv"
    }
}

print("Cấu hình đường dẫn files:")
for source, files in FILES_CONFIG.items():
    print(f"\n{source}:")
    for file_type, path in files.items():
        exists = "✓" if path.exists() else "✗"
        print(f"  {file_type}: {path.name} [{exists}]")

Cấu hình đường dẫn files:

Amazon:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

Hotels:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

AmazonsElectronics:
  all: neutral_output_aspect_filledNone.csv [✓]


In [7]:
def load_and_standardize_data(file_path):
    """
    Đọc file CSV và chuẩn hóa cột thành reviewText, AspectTerm, Sentiment
    
    Parameters:
    - file_path: đường dẫn file
    
    Returns:
    - DataFrame đã chuẩn hóa với 3 cột: reviewText, AspectTerm, Sentiment
    """
    try:
        df = pd.read_csv(file_path)
        
        # Chuẩn hóa tên cột thành chữ thường để dễ xử lý
        df.columns = df.columns.str.strip().str.lower()
        
        # Map các tên cột có thể khác nhau về tên chuẩn
        column_mapping = {
            'reviewtext': 'reviewText',
            'review': 'reviewText',
            'text': 'reviewText',
            'aspectterm': 'AspectTerm',
            'aspect': 'AspectTerm',
            'sentiment': 'Sentiment'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Chỉ giữ 3 cột cần thiết
        required_cols = ['reviewText', 'AspectTerm', 'Sentiment']
        available_cols = [col for col in required_cols if col in df.columns]
        df = df[available_cols]
        
        # Điền 'None' vào các AspectTerm trống hoặc null
        if 'AspectTerm' in df.columns:
            # Đếm số lượng trước khi điền
            empty_mask = df['AspectTerm'].isna() | (df['AspectTerm'].astype(str).str.strip() == '')
            empty_count = empty_mask.sum()
            
            # Điền 'None' vào các ô trống
            df.loc[empty_mask, 'AspectTerm'] = 'None'
            
            print(f"  ✓ Loaded {len(df)} rows from {file_path.name} (filled {empty_count} empty AspectTerm with 'None')")
        else:
            print(f"  ✓ Loaded {len(df)} rows from {file_path.name}")
        
        return df
        
    except Exception as e:
        print(f"  ✗ Error loading {file_path}: {e}")
        return pd.DataFrame()

print("Hàm load_and_standardize_data đã được định nghĩa!")

Hàm load_and_standardize_data đã được định nghĩa!


In [8]:
def merge_all_datasets(files_config, output_path=None, balance_config=None):
    """
    Gộp tất cả các dataset thành một file duy nhất
    
    Parameters:
    - files_config: dict chứa cấu hình các file
    - output_path: đường dẫn lưu file output (optional)
    - balance_config: dict để giới hạn số lượng mẫu theo sentiment (optional)
                      Ví dụ: {'positive': 5000, 'negative': 5000, 'neutral': 5000}
                      Nếu None, giữ nguyên tất cả dữ liệu
    
    Returns:
    - DataFrame đã gộp với 3 cột: reviewText, AspectTerm, Sentiment
    """
    all_dataframes = []
    
    print("=" * 60)
    print("BẮT ĐẦU GỘP DATASETS")
    print("=" * 60)
    
    for source_name, files in files_config.items():
        print(f"\n📂 Loading from {source_name}:")
        for file_type, file_path in files.items():
            if file_path.exists():
                df = load_and_standardize_data(file_path)
                if not df.empty:
                    all_dataframes.append(df)
            else:
                print(f"  ⚠ File not found: {file_path}")
    
    if not all_dataframes:
        print("\n❌ Không có dữ liệu nào được load!")
        return pd.DataFrame()
    
    # Gộp tất cả
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    
    # Áp dụng balance_config nếu có
    if balance_config:
        print("\n" + "=" * 60)
        print("ÁP DỤNG CÂN BẰNG DỮ LIỆU")
        print("=" * 60)
        
        balanced_dfs = []
        for sentiment, max_count in balance_config.items():
            sentiment_df = merged_df[merged_df['Sentiment'].str.lower() == sentiment.lower()]
            current_count = len(sentiment_df)
            
            if current_count > max_count:
                sentiment_df = sentiment_df.sample(n=max_count, random_state=42)
                print(f"  {sentiment}: {current_count:,} → {max_count:,} (giảm {current_count - max_count:,})")
            else:
                print(f"  {sentiment}: {current_count:,} (giữ nguyên, ít hơn {max_count:,})")
            
            balanced_dfs.append(sentiment_df)
        
        merged_df = pd.concat(balanced_dfs, ignore_index=True)
        merged_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ CUỐI CÙNG")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    print(f"\n📋 Các cột: {list(merged_df.columns)}")
    
    # Lưu file nếu có output_path
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"\n💾 Đã lưu file: {output_path}")
    
    return merged_df

print("Hàm merge_all_datasets đã được định nghĩa!")

Hàm merge_all_datasets đã được định nghĩa!


In [9]:
# ============================================
# CẤU HÌNH SỐ LƯỢNG MẪU THEO SENTIMENT
# ============================================
# Chỉnh sửa các giá trị bên dưới để thay đổi số lượng mẫu cho mỗi nhãn
# Đặt None nếu muốn giữ tất cả dữ liệu không giới hạn

BALANCE_CONFIG = {
    'positive': 6600,   # Số lượng mẫu positive tối đa
    'negative': 6600,   # Số lượng mẫu negative tối đa  
    'neutral': 5000    # Số lượng mẫu neutral tối đa
}

# Đặt BALANCE_CONFIG = None nếu muốn giữ tất cả dữ liệu
# BALANCE_CONFIG = None

# ============================================
# Thực hiện gộp dataset
OUTPUT_PATH = BASE_PATH / "merged_dataset_all_13200rows.csv"

merged_df = merge_all_datasets(
    FILES_CONFIG, 
    output_path=OUTPUT_PATH,
    balance_config=BALANCE_CONFIG
)

BẮT ĐẦU GỘP DATASETS

📂 Loading from Amazon:
  ✓ Loaded 6297 rows from train_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 974 rows from dev_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 966 rows from test_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')

📂 Loading from Hotels:
  ✓ Loaded 10126 rows from train_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 1036 rows from dev_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 242 rows from test_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')

📂 Loading from AmazonsElectronics:
  ✓ Loaded 5000 rows from neutral_output_aspect_filledNone.csv (filled 3788 empty AspectTerm with 'None')

KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)

📊 Tổng số mẫu: 24,641

📈 Phân bố theo Sentiment:
Sentiment
positive    15937
neutral      5576
negative     3116

ÁP DỤNG CÂN BẰNG DỮ LIỆU
  positive: 15,937 → 6,600 (giảm 

In [10]:
# Xem một số mẫu dữ liệu
print("📋 Một số mẫu dữ liệu đầu tiên:")
merged_df.head(10)

📋 Một số mẫu dữ liệu đầu tiên:


,reviewText,AspectTerm,Sentiment
0,I have seen and purchased these at lower price...,None,neutral
1,"I was thinking about other thing, it was very ...",None,neutral
2,"There are Better Options than this tired , out...",hotel quality,negative
3,"I think is good, but I think creative labs pro...",None,neutral
4,"This is NOT an Apple product. Although, it is ...",None,neutral
5,"Basically , you end up paying well over 2 time...",food prices,negative
6,"There ' s no bitter after taste , and it ' s g...",food quality,positive
7,"I like the fact that there are no additives , ...",food quality,positive
8,Restaurant very odd .,facilities general,negative
9,Appears to do the trick for being able to char...,price,neutral


## Tổng kết

File đã được tạo với cấu trúc 3 cột:
- `reviewText`: Nội dung review
- `AspectTerm`: Aspect term được trích xuất
- `Sentiment`: Nhãn sentiment (positive/negative/neutral)